In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from fbprophet import Prophet
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
data = pd.read_csv('../input/temperature-change/Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding='latin-1')


In [4]:
data

In [5]:
data.info()

In [6]:
data.isnull().sum()

In [7]:
# preprocessing

In [8]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Remove the standard deviation examples
    df = df.query("Element == 'Temperature change'")
    df = df.groupby('Area').mean()
    df = df.loc[:, 'Y1961':]
    df = pd.DataFrame(df.mean()).reset_index(drop=False)
    df.columns=['ds','y']
    df['ds']=df['ds'].apply(lambda x:x[1:]).astype(np.int)
    
    return df


In [9]:
time_series=preprocess_inputs(data)

In [10]:
time_series

In [11]:
fig=px.line(
time_series,
 x='ds',y='y',
labels={'ds': "Year",'y':"Change In Temprature(°C)"},
)
fig.show()

In-Sample Forecast¶

In [12]:
time_train= time_series.iloc[:44,:].copy()
time_test= time_series.iloc[44:, :].copy()


In [13]:
time_test

In [14]:
in_model=Prophet()
in_model.fit(time_test)

In [15]:
in_forecast=in_model.predict(time_test).loc[:,['ds','yhat']]
in_forecast['ds']=in_forecast['ds'].apply(lambda x:x.year)
in_forecast

In [16]:
in_result=time_series.merge(in_forecast,on='ds',how='left')
fig=px.line(
in_result,x='ds',y=['y','yhat'],
 color_discrete_sequence=['black','red'],
labels={'ds': "Year"},
title="Average Global Temperature change over Time"
)
fig.show()

**Out-of-Sample Forecast**

In [17]:
future_df=pd.DataFrame(np.arange(2020,2046),columns=['ds'])
future_df

In [18]:
out_model=Prophet()
out_model.fit(time_series)

In [19]:
out_forecast=out_model.predict(future_df).loc[:,['ds','yhat']]
out_forecast['ds'] = out_forecast['ds'].apply(lambda x: x.year)
out_forecast

In [20]:
out_result_df=pd.concat([time_series,out_forecast],axis=0)

In [21]:

out_result_df

In [22]:
out_result_df = pd.concat([time_series, out_forecast], axis=0)

fig = px.line(
    out_result_df,
    x='ds',
    y=['y', 'yhat'],
    color_discrete_sequence=['black', 'red'],
    labels={'ds': "Year"},
    title="Average Global Temperature Change Over Time"
)

fig.show()